# Project nº IV: GeoSpatial Project

## PART 1: FILTERING

### Importing libraries

In [208]:
from pymongo import MongoClient, GEO2D
from bson import json_util
import pandas as pd
import json

In [2]:
client = MongoClient()

In [3]:
db = client.get_database("companies")

### Exploring

#### Companies that have both latitude & longitude

In [4]:
res = list(db.companies.find({"offices.latitude":{"$ne":None}, "offices.longitude":{"$ne":None}},{"name":1,"offices":1}))

In [5]:
len(res)

14082

In [6]:
res[:3]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'offices': [{'description': 'Headquarters',
    'address1': '4900 Hopyard Rd.',
    'address2': 'Suite 310',
    'zip_code': '94588',
    'city': 'Pleasanton',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 37.692934,
    'longitude': -121.904945}]},
 {'_id': ObjectId('52cdef7c4bab8bd675297d92'),
  'name': 'Flektor',
  'offices': [{'description': None,
    'address1': '8536 National Blvd, Suite A',
    'address2': None,
    'zip_code': '90232',
    'city': 'Culver City',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 34.025958,
    'longitude': -118.379768}]},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'offices': [{'description': 'Headquarters',
    'address1': '4900 Hopyard Rd',
    'address2': 'Suite 310',
    'zip_code': '94588',
    'city': 'Pleasanton',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 37.692934,
    'longitude': 

#### Different types of categories

In [7]:
cat = list(db.companies.find({"category_code":{"$ne":None}}))

In [8]:
len(cat)

16050

In [9]:
set([document["category_code"] for document in cat])

{'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web'}

## $unwind of the offices

In [58]:
res = db.companies.aggregate([
                      {"$unwind":"$offices"},
                      {"$match":{"offices.latitude":{"$ne":None}}},
                      {"$match":{"offices.longitude":{"$ne":None}}},
                      {"$project":{"_id":0}}
                     ])

In [59]:
res

In [60]:
db.companies_unwinded.insert_many(res)

In [61]:
offices = db.companies_unwinded

## Filter 1: Design

In [119]:
design = list(offices.find({"category_code":"design"},{"name":1,"offices":1}))

In [120]:
design

[{'_id': ObjectId('5fb1638e247255c14a44fb3d'),
  'name': '99designs',
  'offices': {'description': 'United States (HQ)',
   'address1': '447 Battery St.',
   'address2': '3rd Floor',
   'zip_code': '94111',
   'city': 'San Francisco',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.7955307,
   'longitude': -122.4005983}},
 {'_id': ObjectId('5fb1638e247255c14a44fb3e'),
  'name': '99designs',
  'offices': {'description': 'Australia',
   'address1': '204 Wellington St.',
   'address2': '',
   'zip_code': '3065',
   'city': 'Collingwood',
   'state_code': None,
   'country_code': 'AUS',
   'latitude': -37.8026587,
   'longitude': 144.9868546}},
 {'_id': ObjectId('5fb1638e247255c14a44fb3f'),
  'name': '99designs',
  'offices': {'description': 'Europe',
   'address1': 'Schlesische Str. 29-30',
   'address2': '',
   'zip_code': '',
   'city': 'Berlin',
   'state_code': None,
   'country_code': 'DEU',
   'latitude': 52.4986199,
   'longitude': 13.4469031}},
 {'_id': ObjectId

### Key takeaway:
I am interested in being near SF, Collingwood, Berlin, and Ellensburg.

## $near
- 1st: longitude, 2nd: latitude

In [121]:
res = offices.find({},{"name":1,"offices":1})

In [122]:
for c in res:
    filt = {"_id":c["_id"]}
    coord = {
              "type": "Point",
              "coordinates": [c["offices"]["longitude"], c["offices"]["latitude"]]
            }
    update = {"$set":{"coord":coord}}
    offices.update_one(filt,update)

I create geoindex in MongoCompass

In [123]:
# offices.create_index([("coord","2dsphere")])

SanFran

In [138]:
point = {"type":"Point",
         "coordinates":[-122.4005983, 37.7955307]}

query ={
    "coord":{
        "$near":{
            "$geometry":point,
            "$maxDistance":100_000,
        }
    }
}

res_sf = list(offices.find(query))

In [125]:
len(res_sf)

2044

Australia

In [139]:
point = {"type":"Point",
         "coordinates":[144.9868546, 37.7955307]}

query ={
    "coord":{
        "$near":{
            "$geometry":point,
            "$maxDistance":100_000,
        }
    }
}

res_aus = list(offices.find(query))

In [127]:
len(res_aus)

0

In [140]:
point = {"type":"Point",
         "coordinates":[144.9868546, 37.7955307]}

query ={
    "coord":{
        "$near":{
            "$geometry":point,
            "$maxDistance":1_000_000,
        }
    }
}

res_aus = list(offices.find(query))

In [129]:
len(res_aus)

9

Berlin

In [141]:
point = {"type":"Point",
         "coordinates":[13.4469031, 52.4986199]}

query ={
    "coord":{
        "$near":{
            "$geometry":point,
            "$maxDistance":100_000,
        }
    }
}

res_ber = list(offices.find(query))

In [131]:
len(res_ber)

77

Ellensburg

In [142]:
point = {"type":"Point",
         "coordinates":[-122.6931439, 45.7968414]}

query ={
    "coord":{
        "$near":{
            "$geometry":point,
            "$maxDistance":100_000,
        }
    }
}

res_ell = list(offices.find(query))

In [133]:
len(res_ell)

73

### 1st observation:
SF is the city with the highest number of companies within a design environment.

In [143]:
comp_design = res_sf + res_aus + res_ber + res_ell
len(comp_design)

2203

In [144]:
type(comp_design)

list

converting into a JSON and inserting into db

In [145]:
db.companies_unwinded_des.insert_many(comp_design)

## Filter 2: Tech

In [171]:
offices_2 = db.companies_unwinded_des

In [172]:
offices_2

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies_unwinded_des')

In [178]:
res_2 = offices_2.find({},{"name":1,"offices":1})

In [179]:
len(list(res_2))

2203

creating geoIndex in new db

In [148]:
for c in res_2:
    filt = {"_id":c["_id"]}
    coord = {
              "type": "Point",
              "coordinates": [c["offices"]["longitude"], c["offices"]["latitude"]]
            }
    update = {"$set":{"coord":coord}}
    offices_2.update_one(filt,update)

Filter tech:
- by definition must be a tech company
- total money raised > $1M
- nº employees >50, <250

In [192]:
filter_tech = {"category_code":{"$regex":"tech$"},
               "total_money_raised":{"$gte":"$1_000_000"},
               "number_of_employees":{"$gte":50, "$lte":250}}

one_mil = list(db.companies_unwinded_des.find(filter_tech))

In [194]:
len(one_mil)

6

### 2nd observation:
There are 6 companies near to tech startups and within a design environment.

In [196]:
db.companies_unwinded_des_tech.insert_many(one_mil)

In [206]:
offices_3 = list(db.companies_unwinded_des_tech.find(filter_tech,{"offices":1}))

In [207]:
offices_3

[{'_id': ObjectId('5fb1638f247255c14a451e65'),
  'offices': {'description': 'Headquarters',
   'address1': '46400 Fremont Boulevard',
   'address2': '',
   'zip_code': '94538',
   'city': 'Fremont',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.790153,
   'longitude': -122.398669}},
 {'_id': ObjectId('5fb1638f247255c14a451aea'),
  'offices': {'description': 'San Francisco HQ',
   'address1': '45 Fremont Street, 32nd Floor',
   'address2': '',
   'zip_code': '94105',
   'city': 'San Francisco',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.7871306,
   'longitude': -122.4041075}},
 {'_id': ObjectId('5fb1638f247255c14a451d0b'),
  'offices': {'description': 'Headquarters',
   'address1': '260 Littlefield Avenue',
   'address2': '',
   'zip_code': '94080',
   'city': 'South San Francisco',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.645401,
   'longitude': -122.394464}},
 {'_id': ObjectId('5fb1638f247255c14a451993'),
  'office

### 3rd observation:
All the offices are in CA.

### Saving mongodb query to json file

In [213]:
res = db.companies_unwinded_des_tech.find(filter_tech)

In [214]:
res = list(res)

In [215]:
with open("data/comp_selec.json", "w+") as file:
    file.write(json_util.dumps(res))

In [218]:
pd.read_json("data/comp_selec.json")

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners,coord
0,{'$oid': '5fb1638f247255c14a451e65'},GreenVolts,greenvolts,http://www.crunchbase.com/company/greenvolts,http://www.greenvolts.com,,,GreenVolts,cleantech,68,...,None,[],"{'description': 'Headquarters', 'address1': '4...","[{'id': 11069, 'description': 'GreenVolts Look...",NaN,[],[],[],[],"{'type': 'Point', 'coordinates': [-122.398669,..."
1,{'$oid': '5fb1638f247255c14a451aea'},Sunrun,sunrun,http://www.crunchbase.com/company/sunrun,http://www.sunrun.com,http://www.sunrun.com/blog,,Sunrun,cleantech,180,...,None,[],"{'description': 'San Francisco HQ', 'address1'...","[{'id': 9626, 'description': 'Solar Co. SunRun...",NaN,[],"[{'available_sizes': [[[150, 117], 'assets/ima...",[],[],"{'type': 'Point', 'coordinates': [-122.4041075..."
2,{'$oid': '5fb1638f247255c14a451d0b'},Catalyst Biosciences,catalyst-biosciences,http://www.crunchbase.com/company/catalyst-bio...,http://www.catalystbiosciences.com,,,,biotech,50,...,None,[],"{'description': 'Headquarters', 'address1': '2...",[],NaN,[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],"{'type': 'Point', 'coordinates': [-122.394464,..."
3,{'$oid': '5fb1638f247255c14a451993'},eMeter,emeter,http://www.crunchbase.com/company/emeter,http://www.emeter.com,http://www.emeter.com/category/blog/,http://www.emeter.com/?feed=rss2,emeter,cleantech,200,...,"{'price_amount': None, 'price_currency_code': ...",[],"{'description': 'Corporate Headquarters', 'add...",[],NaN,[],"[{'available_sizes': [[[141, 150], 'assets/ima...",[],[],"{'type': 'Point', 'coordinates': [-122.284114,..."
4,{'$oid': '5fb1638f247255c14a451b34'},TRIRIGA,tririga,http://www.crunchbase.com/company/tririga,http://www.tririga.com,,,TRIRIGAINC,cleantech,214,...,"{'price_amount': None, 'price_currency_code': ...",[],"{'description': 'Pleasanton', 'address1': '599...","[{'id': 14398, 'description': 'ARMONK, N.Y. --...",NaN,[],[],[{'external_url': 'http://www.environmentallea...,[],"{'type': 'Point', 'coordinates': [-121.9031911..."
5,{'$oid': '5fb1638f247255c14a451810'},SolarEdge,solaredge,http://www.crunchbase.com/company/solaredge,http://www.solaredge.com,http://www.solaredge.com/groups/blog,,SolarEdgePV,cleantech,230,...,None,[],"{'description': 'Local US Office', 'address1':...",[],NaN,"[{'embed_code': '<object width=""430"" height=""2...",[],[],[],"{'type': 'Point', 'coordinates': [-121.9382415..."
